#Import libraies and packages

In [1]:
pip install ray torch torchvision

     |████████████████████████████████| 53.2 MB 263 kB/s 
     |████████████████████████████████| 4.1 MB 34.9 MB/s 
     |████████████████████████████████| 144 kB 34.4 MB/s 
     |████████████████████████████████| 8.8 MB 33.6 MB/s 
     |████████████████████████████████| 461 kB 36.2 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.46.1
    Uninstalling grpcio-1.46.1:
      Successfully uninstalled grpcio-1.46.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [2]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=5a2237c593b69fcfe4420ab89543e92d71e2a870a7cd47dd1e43f453974a5e1b
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [3]:
# # Solucion par ael error int-float
# import psutil
# import ray
# ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total

In [4]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pandas as pd
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model

In [7]:
import tensorflow as tf

def augment_images(x, config):
    if config['use_contrast'] == "True":
        x = tf.keras.layers.experimental.preprocessing.RandomContrast(
            config['contrast_factor']
        )(x)

    if config['use_rotation'] == "True":
        x = tf.keras.layers.experimental.preprocessing.RandomRotation(
            config['rotation_factor']
        )(x)
    
    if config['use_flip'] == "True":
        x = tf.keras.layers.experimental.preprocessing.RandomFlip(
            config['flip_mode']
        )(x)

    return x

def ConvNet(config, len_classes=2):
    input = tf.keras.layers.Input(shape=(20, 20, 1))
    # x = augment_images(input, config)
    x = input
    x = tf.keras.layers.Conv2D(filters=config['conv_block1_filters'], kernel_size=3, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters=config['conv_block2_filters'], kernel_size=3, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)


    x = tf.keras.layers.Conv2D(filters=config['conv_block3_filters'], kernel_size=3, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)


    x = tf.keras.layers.Conv2D(filters=config['conv_block4_filters'], kernel_size=2, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters=config['conv_block5_filters'], kernel_size=1, strides=1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    if config['fc_layer_type'] == 'dense':
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling2D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling2D()(x)

        # Fully connected layer 1
        x = tf.keras.layers.Dense(units=config['fc1_units'])(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)

        # Fully connected layer 2
        x = tf.keras.layers.Dense(units=len_classes)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        predictions = tf.keras.layers.Dense(1,tf.keras.layers.Activation('sigmoid'))(x)

    else:
        # Fully connected layer 1
        x = tf.keras.layers.Conv2D(filters=config['fc1_units'], kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)


        # Fully connected layer 2
        x = tf.keras.layers.Conv2D(filters=len_classes, kernel_size=1, strides=1)(x)
        x = tf.keras.layers.Dropout(config['dropout_rate'])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        
        if config['pool_type'] == 'max':
            x = tf.keras.layers.GlobalMaxPooling2D()(x)
        else:
            x = tf.keras.layers.GlobalAveragePooling2D()(x)
        
        predictions = tf.keras.layers.Dense(1,tf.keras.layers.Activation('sigmoid'))(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model
   

## Metrics

In [8]:
# from keras import backend as K

# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Training function

In [26]:
from sklearn import preprocessing

def train_mnist( config):
  path ='/content/drive/MyDrive/UPM/Clinical_data_and_RNA_total_Features_PFS.csv'
  data_frame = pd.read_csv(path)
  # data_frame = data_frame.set_index("gene_name")
  # data_frame=data_frame.T
  # data_frame.head()
  # data_frame=data_frame.iloc[np.repeat(np.arange(len(data_frame)), 3)]
  
  from sklearn.model_selection import train_test_split
  X = data_frame[['5S_rRNA', 'AADACL2-AS1', 'AB015752.3', 'AB019438.66', 'ABHD17AP7', 'AC000077.2', 'AC000111.3', 'AC000111.5', 'AC000124.1', 'AC002368.4', 'AC002472.11', 'AC002486.3', 'AC002542.2', 'AC003084.2', 'AC003682.16', 'AC003682.17', 'AC003986.5', 'AC004006.2', 'AC004012.1', 'AC004014.3', 'AC004066.3', 'AC004112.4', 'AC004158.3', 'AC004221.2', 'AC004449.6', 'AC004510.3', 'AC004549.6', 'AC004862.6', 'AC004870.3', 'AC004941.3', 'AC004947.2', 'AC005008.2', 'AC005094.2', 'AC005178.1', 'AC005197.2', 'AC005235.1', 'AC005307.1', 'AC005355.1', 'AC005387.3', 'AC005392.13', 'AC005498.4', 'AC005518.2', 'AC005550.5', 'AC005597.1', 'AC005624.2', 'AC005722.4', 'AC005740.5', 'AC006003.3', 'AC006007.1', 'AC006041.1', 'AC006116.14', 'AC006130.4', 'AC006153.3', 'AC006296.2', 'AC006372.4', 'AC006372.5', 'AC006372.6', 'AC006373.1', 'AC006427.2', 'AC006499.1', 'AC006499.3', 'AC006499.5', 'AC006499.6', 'AC006509.4', 'AC006960.7', 'AC006987.5', 'AC007128.1', 'AC007131.2', 'AC007163.6', 'AC007193.9', 'AC007249.3', 'AC007253.1', 'AC007292.7', 'AC007322.1', 'AC007349.7', 'AC007391.2', 'AC007403.3', 'AC007463.2', 'AC007557.4', 'AC007563.3', 'AC007563.5', 'AC007568.1', 'AC007679.3', 'AC007682.1', 'AC007740.1', 'AC007742.3', 'AC007796.1', 'AC007879.1', 'AC007879.3', 'AC008067.2', 'AC008074.4', 'AC008154.4', 'AC008278.2', 'AC008281.1', 'AC008746.10', 'AC008746.9', 'AC008834.2', 'AC008940.1', 'AC009161.1', 'AC009237.16', 'AC009237.17', 'AC009238.7', 'AC009238.8', 'AC009310.1', 'AC009312.1', 'AC009313.1', 'AC009410.1', 'AC009477.7', 'AC009487.4', 'AC009487.5', 'AC009541.1', 'AC009970.1', 'AC010096.2', 'AC010141.1', 'AC010518.3', 'AC010729.1', 'AC010729.2', 'AC010731.2', 'AC010731.6', 'AC010884.1', 'AC010907.5', 'AC010967.1', 'AC010967.2', 'AC010974.3', 'AC010983.1', 'AC010987.6', 'AC011196.3', 'AC011243.1', 'AC011286.1', 'AC011343.1', 'AC011516.2', 'AC011524.3', 'AC011648.1', 'AC011742.5', 'AC012074.2', 'AC012075.2', 'AC012322.1', 'AC012354.6', 'AC012363.13', 'AC012456.4', 'AC012506.1', 'AC012506.2', 'AC012506.3', 'AC012506.4', 'AC012507.3', 'AC012507.4', 'AC012513.6', 'AC012668.1', 'AC013402.5', 'AC013429.4', 'AC013448.2', 'AC013480.2', 'AC013733.3', 'AC015936.3', 'AC016694.2', 'AC016735.1', 'AC016745.3', 'AC016831.3', 'AC016831.5', 'AC016912.3', 'AC017019.1', 'AC017048.2', 'AC018359.1', 'AC018643.4', 'AC018685.1', 'AC018717.1', 'AC018799.1', 'AC018804.3', 'AC018880.1', 'AC018880.2', 'AC018892.3', 'AC019064.1', 'AC019118.4', 'AC020595.1', 'AC020743.2', 'AC020915.2', 'AC022201.4', 'AC023115.4', 'AC023128.1', 'AC024082.4', 'AC024704.2', 'AC026202.5', 'AC026954.6', 'AC027124.2', 'AC034228.2', 'AC060834.3', 'AC062016.1', 'AC062022.1', 'AC063976.1', 'AC063976.3', 'AC064834.1', 'AC064834.2', 'AC067945.2', 'AC067956.1', 'AC068057.1', 'AC068490.1', 'AC068492.1', 'AC068535.3', 'AC068831.3', 'AC069155.1', 'AC073218.1', 'AC073464.6', 'AC073641.2', 'AC074338.4', 'AC078842.3', 'AC078882.1', 'AC079112.1', 'AC079117.1', 'AC079145.4', 'AC079154.1', 'AC079325.6', 'AC079586.1', 'AC079610.1', 'AC079776.1', 'AC079987.2', 'AC083864.3', 'AC083939.1', 'AC090018.1', 'AC090952.5', 'AC090957.2', 'AC091133.1', 'AC091729.8', 'AC091814.2', 'AC092155.4', 'AC092159.3', 'AC092168.3', 'AC092566.1', 'AC092570.1', 'AC092570.2', 'AC092578.1', 'AC092601.1', 'AC092635.1', 'AC092657.2', 'AC092933.4', 'AC093063.2', 'AC093084.1', 'AC093106.4', 'AC093326.1', 'AC093381.2', 'AC093590.1', 'AC093642.6', 'AC096582.9', 'AC096669.2', 'AC096732.2', 'AC097468.7', 'AC097713.3', 'AC097713.4', 'AC098592.6', 'AC098828.3', 'AC098872.3', 'AC098973.1', 'AC099344.2', 'AC099344.3', 'AC099552.3', 'AC102948.2', 'AC103563.5', 'AC103563.7', 'AC103563.8', 'AC104076.3', 'AC104306.4', 'AC104777.2', 'AC104777.4', 'AC107083.1', 'AC108032.1', 'AC108066.1', 'AC109589.1', 'AC110620.1', 'AC110754.3', 'AC112198.1', 'AC112220.2', 'AC112518.3', 'AC113167.1', 'AC113167.2', 'AC113607.2', 'AC114752.3', 'AC114814.2', 'AC114877.3', 'AC118653.2', 'AC123900.2', 'AC124057.5', 'AC125421.1', 'AC128709.2', 'AC128709.3', 'AC128709.4', 'AC133633.1', 'AC133680.1', 'AC133785.1', 'AC140481.8', 'AC142119.1', 'AC142293.3', 'AC144450.1', 'AC144833.1', 'ADI1P2', 'AE000661.36', 'AF186192.6', 'AHCYP3', 'AL133249.1', 'AP000345.4', 'AP000356.2', 'AP000619.5', 'AP000648.6', 'AP000997.2', 'AP006285.7', 'ATP5F1P4', 'ATP5G1P8', 'BAATP1', 'BOK-AS1', 'BRI3BPP1', 'BSN-AS1', 'BSN-AS2', 'CASC16', 'CDCA4P3', 'CDKN2AIPNLP2', 'CICP1', 'CLUHP4', 'CNN2P2', 'CTA-392E5.1', 'CTB-138E5.1', 'CTB-164N12.1', 'CTB-49A3.1', 'CTC-304I17.5', 'CTC-339O9.2', 'CTC-501O10.1', 'CTD-2130F23.2', 'CTD-2269E23.3', 'CTD-2308B18.1', 'CTD-2319I12.5', 'CTD-2555O16.1', 'CTD-3187F8.11', 'CYCSP26', 'CYP2T3P', 'DDX43P2', 'DLG3-AS1', 'DPRXP6', 'GLUD1P4', 'GLULP6', 'GRAMD4P7', 'GS1-114I9.1', 'GS1-251I9.2', 'GTSCR1', 'HNRNPH3P1', 'IFT74-AS1', 'IGKV7-3', 'KRT8P6', 'KRTAP20-3', 'LINC00309', 'LINC00364', 'LINC00391', 'LINC00466', 'LINC00838', 'LINC00856', 'LINC00864', 'LINC00879', 'LINC01326', 'LINC01524', 'LRRC34P1', 'MARCKSL1P2', 'MIR1273F', 'MIR4290HG', 'MRLN', 'MTATP6P23', 'MTCO2P16', 'NHEG1', 'NOS2P4', 'OR10B1P', 'OR13Z2P', 'OR2AD1P', 'OR2B8P', 'OR5BN2P', 'OSBPL9P1', 'PDX1-AS1', 'PGBD4P5', 'RN7SKP238', 'RNA5SP205', 'RNF212', 'RNU6-1090P', 'RP1-151B14.9', 'RP11-100K18.1', 'RP11-107N7.1', 'RP11-123J14.1', 'RP11-203B7.2', 'RP11-2H8.3', 'RP11-307O10.1', 'RP11-359G22.2', 'RP11-361A23.3', 'RP11-378I6.1', 'RP11-430C1.1', 'RP11-434D11.4', 'RP11-438B23.2', 'RP11-520D19.2', 'RP11-661A12.12', 'RP11-846C15.2', 'RP11-97F8.1', 'RP13-631K18.2', 'RP3-406P24.4', 'RP4-581O6.1', 'RP5-998G20.1', 'RPL35AP', 'XXyac-YR12DB5.1', 'YBX1P9']].astype(float)
  Y=[]
  for i in range (len(data_frame)):
      if data_frame.Target[i]=='NR': # If PFS is lower than 3 months, I will consider it as NonResponder (NR)
          Y.append(0)
      else:
          Y.append(1)# If PFS is over 3 months, I will consider it as Responder (R)
  scaler = preprocessing.MinMaxScaler()
  names = X.columns
  d = scaler.fit_transform(X)
  X = pd.DataFrame(d, columns=names)
  # scaled_df.head()
  # for i in X[:0]:
  #   # print(X[i])
  #   X[i] = X[i] /X[i].abs().max() 
  XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size=0.20, random_state=125)
  # Convert sets to arrays
  XTrain = XTrain.values
  XTest = XTest.values
  # print("hola")
  # XTrain.to_xarray()
  # XTest.to_xarray()
  # It is mandatory to transform Y list into array for trainning the model
  yTrain=np.array(yTrain)
  yTest=np.array(yTest)

  # yTrain=yTrain.to_xarray()
  # yTest=yTest.to_xarray()
  # building the input vector from the 28x28 pixels
  X_train = XTrain.reshape(XTrain.shape[0], 20 , 20, 1)
  X_test = XTest.reshape(XTest.shape[0], 20, 20, 1)
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
# Create FCN model
  model = ConvNet(config)

  # Compile model with losses and metrics
  # tensorflow.keras.optimizers.RMSprop(learning_rate = 0.0001), metrics=['accuracy'])
  # tf.keras.optimizers.Nadam(learning_rate
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate =config['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'#, f1_m,precision_m, recall_m
                           ])
  
  # Create callbacks to be used during model training
  # callbacks = create_callbacks()#self.final_run, self.snapshot_dir)
  print("hhhhhhhhhhhhhhhhhhh")
  # Start model training
  history_m = model.fit(X_train, yTrain,
                      # steps_per_epoch=len(X_train),
                      epochs=1,
                      # callbacks=callbacks,
                      validation_data=(X_test, yTest),
                      # validation_steps=len(yTrain)
                      )
  # print(type(history_m["loss"]))
  print("tipado")
  print(type(history_m))
  # print(history_m)
  print("diccionario")
  print(history_m.history)
  # history_m = yield from history_m.items()
  print(history_m.history["val_loss"][0])
  print(type(history_m.history["val_loss"][0]))
  history_m = {
  "loss": history_m.history["loss"][0],
  "val_loss": history_m.history["val_loss"][0],
  "accuracy": history_m.history["accuracy"][0],
  "val_accuracy": history_m.history["val_accuracy"][0]
  }
  return history_m

# Search space


In [10]:
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch

search_space = {"lr": hp.choice("lr", [0.0001, 0.001, 0.01, 0.1]),
                    "batch_size": hp.choice("batch_size", [8, 16, 32, 64]), 
                    # "use_contrast": hp.choice("use_contrast", ["True", "False"]),
                    # "contrast_factor": hp.choice('contrast_factor', [0.1, 0.2, 0.3, 0.4]),
                    # "use_rotation": hp.choice("use_rotation", ["True", "False"]),
                    # "rotation_factor": hp.choice('rotation_factor', [0.1, 0.2, 0.3, 0.4]),
                    # "use_flip": hp.choice("use_flip", ["True", "False"]),
                    # "flip_mode": hp.choice('flip_mode', ["horizontal", "vertical"]),
                    "dropout_rate": hp.choice("dropout_rate", [0.1, 0.2, 0.3, 0.4, 0.5]),
                    "conv_block1_filters":hp.choice("conv_block1_filters", [32, 64, 128, 256, 512]),
                    "conv_block2_filters":hp.choice("conv_block2_filters", [32, 64, 128, 256, 512]),
                    "conv_block3_filters":hp.choice("conv_block3_filters", [32, 64, 128, 256, 512]),
                    "conv_block4_filters":hp.choice("conv_block4_filters", [32, 64, 128, 256, 512]),
                    "conv_block5_filters":hp.choice("conv_block5_filters", [32, 64, 128, 256]),
                    "fc_layer_type": hp.choice("fc_layer_type", ['dense', 'convolution']),
                    "pool_type": hp.choice("pool_type", ['max', 'average']),
                    "fc1_units":hp.choice("fc1_units", [32, 64, 128, 256, 512])
  }

# hyperopt_search = HyperOptSearch(search_space, metric="mean_accuracy", mode="max")

intial_best_config = [{"lr": 0.1,
                            "batch_size": 8, 
                            # "use_contrast": 1,
                            # "contrast_factor": 0,
                            # "use_rotation": 1,
                            # "rotation_factor": 0,
                            # "use_flip": 1,
                            # "flip_mode": 0,
                            "dropout_rate": 0.1,
                            "conv_block1_filters": 32,
                            "conv_block2_filters": 32,
                            "conv_block3_filters": 32,
                            "conv_block4_filters": 32,
                            "conv_block5_filters": 32,
                            "fc_layer_type": 'dense',
                            "pool_type": 'max',
                            "fc1_units": 32}]

# Bayesian Optimization

In [11]:
scheduler = AsyncHyperBandScheduler(time_attr='training_iteration',
                                            metric="val_accuracy",
                                            mode="max",
                                            grace_period=10)
hyperopt_search = HyperOptSearch(search_space, metric="val_accuracy", mode="max", points_to_evaluate=intial_best_config)
search_alg = ConcurrencyLimiter(hyperopt_search, max_concurrent=1)

In [12]:
analysis = tune.run(train_mnist, num_samples=50, search_alg=search_alg,  scheduler=scheduler, resources_per_trial={'gpu': 1})

2022-05-19 11:18:29,286	WARNING function_runner.py:599 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-05-19 11:18:29,313	INFO logger.py:618 -- pip install "ray[tune]" to see TensorBoard files.
2022-05-19 11:18:29,315	WARNING callback.py:126 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2022-05-19 11:18:29,485	INFO trial_runner.py:803 -- starting train_mnist_68eb5a08


Trial name,status,loc,batch_size,conv_block1_filters,conv_block2_filters,conv_block3_filters,conv_block4_filters,conv_block5_filters,dropout_rate,fc1_units,fc_layer_type,lr,pool_type
train_mnist_68eb5a08,RUNNING,172.28.0.2:600,8,32,32,32,32,32,0.1,32,dense,0.1,max


Trial name,status,loc,batch_size,conv_block1_filters,conv_block2_filters,conv_block3_filters,conv_block4_filters,conv_block5_filters,dropout_rate,fc1_units,fc_layer_type,lr,pool_type
train_mnist_68eb5a08,RUNNING,172.28.0.2:600,8,32,32,32,32,32,0.1,32,dense,0.1,max


Trial name,status,loc,batch_size,conv_block1_filters,conv_block2_filters,conv_block3_filters,conv_block4_filters,conv_block5_filters,dropout_rate,fc1_units,fc_layer_type,lr,pool_type
train_mnist_68eb5a08,RUNNING,172.28.0.2:600,8,32,32,32,32,32,0.1,32,dense,0.1,max


(train_mnist pid=600) 2022-05-19 11:18:53.143380: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


...

(train_mnist pid=4176) Model: "model"
(train_mnist pid=4176) _________________________________________________________________
(train_mnist pid=4176)  Layer (type)                Output Shape              Param #   
(train_mnist pid=4176) =================================================================
(train_mnist pid=4176)  input_1 (InputLayer)        [(None, 20, 20, 1)]       0         
(train_mnist pid=4176)                                                                  
(train_mnist pid=4176)  conv2d (Conv2D)             (None, 18, 18, 64)        640       
(train_mnist pid=4176)                                                                  
(train_mnist pid=4176)  batch_normalization (BatchN  (None, 18, 18, 64)       256       
(train_mnist pid=4176)  ormalization)                                                   
(train_mnist pid=4176)                                                                  
(train_mnist pid=4176)  activation (Activation)     (None, 18, 18, 64)  

Trial name,status,loc,batch_size,conv_block1_filters,conv_block2_filters,conv_block3_filters,conv_block4_filters,conv_block5_filters,dropout_rate,fc1_units,fc_layer_type,lr,pool_type,iter,total time (s),loss,val_loss,accuracy
train_mnist_e0e980b4,RUNNING,172.28.0.2:4176,8,64,256,128,256,32,0.1,256,dense,0.001,average,,,,,
train_mnist_06208298,TERMINATED,172.28.0.2:2893,64,128,512,64,32,64,0.4,64,dense,0.01,average,1,15.7174,0.798599,7.75721,0.548611
train_mnist_0aed775a,TERMINATED,172.28.0.2:1423,16,256,64,64,256,32,0.3,32,convolution,0.01,max,1,15.0433,1.6705,2.67954,0.548611
train_mnist_125ed5d2,TERMINATED,172.28.0.2:2962,8,256,128,128,512,256,0.2,256,convolution,0.001,average,1,15.3659,0.699964,0.671176,0.555556
train_mnist_1701dbda,TERMINATED,172.28.0.2:1495,16,64,512,512,256,128,0.2,64,dense,0.001,max,1,16.1045,0.722589,0.70587,0.541667
train_mnist_1eb5e636,TERMINATED,172.28.0.2:3032,8,256,128,128,512,256,0.2,256,convolution,0.1,average,1,15.1001,0.785071,42271.7,0.444444
train_mnist_2383fb72,TERMINATED,172.28.0.2:1570,16,32,32,256,32,32,0.5,128,dense,0.0001,max,1,15.5674,0.724275,0.693634,0.506944
train_mnist_2b0d3254,TERMINATED,172.28.0.2:3104,8,256,64,128,512,256,0.4,256,convolution,0.001,average,1,15.3671,0.686485,0.694812,0.5625
train_mnist_2fd1ef88,TERMINATED,172.28.0.2:1641,16,512,512,256,256,128,0.1,512,convolution,0.001,max,1,17.4771,0.774419,0.713471,0.520833
train_mnist_3717f4ee,TERMINATED,172.28.0.2:3176,8,256,32,128,512,256,0.3,32,convolution,0.01,average,1,15.6927,0.675158,0.679836,0.590278


5/5 [==============================] - ETA: 0s - loss: 0.7143 - accuracy: 0.6042
Result for train_mnist_e0e980b4:
  accuracy: 0.6041666865348816
  date: 2022-05-19_11-36-29
  done: false
  experiment_id: 9fd59256628841529d096ee472cac0d4
  hostname: 341ac1e1dc5b
  iterations_since_restore: 1
  loss: 0.7142836451530457
  node_ip: 172.28.0.2
  pid: 4176
  time_since_restore: 15.321443319320679
  time_this_iter_s: 15.321443319320679
  time_total_s: 15.321443319320679
  timestamp: 1652960189
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: e0e980b4
  val_accuracy: 0.5675675868988037
  val_loss: 0.6855979561805725
  warmup_time: 0.0039370059967041016
  
Result for train_mnist_e0e980b4:
  accuracy: 0.6041666865348816
  date: 2022-05-19_11-36-29
  done: true
  experiment_id: 9fd59256628841529d096ee472cac0d4
  experiment_tag: 50_batch_size=8,conv_block1_filters=64,conv_block2_filters=256,conv_block3_filters=128,conv_block4_filters=256,conv_block5_filters=32,dropout_rate=0.1,fc1_

Trial name,status,loc,batch_size,conv_block1_filters,conv_block2_filters,conv_block3_filters,conv_block4_filters,conv_block5_filters,dropout_rate,fc1_units,fc_layer_type,lr,pool_type,iter,total time (s),loss,val_loss,accuracy
train_mnist_68eb5a08,TERMINATED,172.28.0.2:600,8,32,32,32,32,32,0.1,32,dense,0.1,max,1,35.1185,0.734559,191.399,0.5
train_mnist_832702aa,TERMINATED,172.28.0.2:685,64,32,32,128,32,256,0.5,32,convolution,0.001,max,1,26.3774,1.44961,0.696208,0.548611
train_mnist_95d26c50,TERMINATED,172.28.0.2:762,64,256,512,32,32,32,0.2,32,dense,0.01,max,1,18.3615,0.797753,1.84715,0.520833
train_mnist_a39f1860,TERMINATED,172.28.0.2:838,16,64,512,256,128,256,0.4,32,convolution,0.0001,max,1,15.549,3.06146,0.692285,0.451389
train_mnist_b02fbfe4,TERMINATED,172.28.0.2:914,8,512,32,64,128,128,0.3,128,dense,0.0001,max,1,16.4584,0.75056,0.693585,0.541667
train_mnist_bd3c0756,TERMINATED,172.28.0.2:987,32,128,64,128,64,32,0.5,128,dense,0.1,average,1,15.897,0.773005,2203.34,0.395833
train_mnist_c9fe012e,TERMINATED,172.28.0.2:1061,64,512,128,256,64,256,0.5,128,dense,0.001,max,1,15.6611,0.794908,0.691936,0.527778
train_mnist_d6a1467a,TERMINATED,172.28.0.2:1136,32,64,128,512,128,64,0.1,64,dense,0.0001,average,1,16.3899,0.705615,0.69325,0.520833
train_mnist_e367a9bc,TERMINATED,172.28.0.2:1207,8,512,128,512,64,256,0.2,512,dense,0.001,average,1,15.8841,0.786473,0.676599,0.527778
train_mnist_f09b7de8,TERMINATED,172.28.0.2:1277,16,32,256,512,128,128,0.3,128,dense,0.0001,average,1,18.5024,0.785218,0.69214,0.479167


2022-05-19 11:36:29,986	INFO tune.py:702 -- Total run time: 1080.70 seconds (1080.47 seconds for the tuning loop).


In [15]:
best_config = analysis.get_best_config(metric="val_accuracy", mode='max')

In [16]:
best_config

{'batch_size': 8,
 'conv_block1_filters': 128,
 'conv_block2_filters': 128,
 'conv_block3_filters': 512,
 'conv_block4_filters': 64,
 'conv_block5_filters': 256,
 'dropout_rate': 0.2,
 'fc1_units': 256,
 'fc_layer_type': 'dense',
 'lr': 0.001,
 'pool_type': 'average'}

In [31]:
history = train_mnist(best_config)

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 20, 20, 1)]       0         
                                                                 
 conv2d_50 (Conv2D)          (None, 18, 18, 128)       1280      
                                                                 
 batch_normalization_70 (Bat  (None, 18, 18, 128)      512       
 chNormalization)                                                
                                                                 
 activation_70 (Activation)  (None, 18, 18, 128)       0         
                                                                 
 conv2d_51 (Conv2D)          (None, 16, 16, 128)       147584    
                                                                 
 batch_normalization_71 (Bat  (None, 16, 16, 128)      512       
 chNormalization)                                         